In [1]:
%cp -r /kaggle/input/ball-track/ /kaggle/working/ball-track
!pip install /kaggle/working/ball-track/pkgs/loguru-0.6.0-py3-none-any.whl

Processing ./ball-track/pkgs/loguru-0.6.0-py3-none-any.whl


In [3]:
from src.yolox.exp import get_exp
from itertools import accumulate
from loguru import logger
from src.yolox.ball_interpolation import bt_smooth_tracking, get_cropped_frames
from src.yolox.inference import Predictor
from src.yolox.utils import get_model_info
import torch, glob, os, cv2, time, numpy as np, pandas as pd


In [4]:
IMAGE_EXT = [".jpg", ".jpeg", ".webp", ".bmp", ".png"]

def get_ball(predictor, config, video_path, start_time=0):
    print(video_path)
    cap = cv2.VideoCapture(video_path)
    border = 8
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_no = 0
    save_path = os.path.join(f'./tracking_outputs/{video_path[video_path.rfind("/")+1:-4]}.mp4')
    print(f"Saving to {save_path}")
    vid_writer = cv2.VideoWriter(
        save_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (config.crop_frame_size[0], config.crop_frame_size[1])
    )
    
    frames, centers, save_centers = [], [], []
    batch_frames = []
    batch_num = 0
    while True:
        ret_val, frame = cap.read()
        if ret_val:
            timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000
            if timestamp >= start_time:
                resized = cv2.resize(frame, (1280, 720))
                frame = cv2.copyMakeBorder(resized, border, border, 0, 0, cv2.BORDER_CONSTANT, None, value = 0)
                frame_shape = frame.shape
                batch_frames.append(frame)
                
                if len(batch_frames) == config.bt_batch_size:
                    outputs, ratio = predictor.batch_inference(batch_frames)
                    frames += batch_frames
                    batch_num += 1
                    if (batch_num % 20) == 0:
                          print(batch_num, timestamp)

                    for output in outputs:

                        if output is not None:
                            output = output.cpu().numpy()
                        if output is not None:
                            center = ((output[0, 2] / ratio + output[0, 0] / ratio) // 2,
                                      (output[0, 1] / ratio + output[0, 3] / ratio) // 2)
                        else:
                            center = None

                        centers.append(center)
                        batch_frames = []


                # if enough frames have been accumulated, smooth ball tracking and get cropped frames
                if len(frames) > 300:
                    bt_centers = bt_smooth_tracking(centers, n_pts=config.bt_smooth_n_pts, outlier_threshold = config.outlier_thresh)
                    bt_centers = list(accumulate(bt_centers, lambda x, y: y or x))
                    bt_centers = [c if c is not None else (frame.shape[1] // 2, frame.shape[0] // 2)
                                  for c in bt_centers]
                    
                    subset_cropped_frames = get_cropped_frames(frames, bt_centers, config)
                    
                    for idx, save_frame in enumerate(subset_cropped_frames):
                        vid_writer.write(save_frame)
                    save_centers += centers
                    frames, centers = [], []
                    
                frame_no += 1
            
        else:
            
            # run last batch
            outputs, ratio = predictor.batch_inference(batch_frames)
            frames += batch_frames

            for output in outputs:

                if output is not None:
                    output = output.cpu().numpy()
                if output is not None:
                    center = ((output[0, 2] / ratio + output[0, 0] / ratio) // 2,
                              (output[0, 1] / ratio + output[0, 3] / ratio) // 2)
                else:
                    center = None

                centers.append(center)
                        
            # get rest of the frames
            bt_centers = bt_smooth_tracking(centers, n_pts=config.bt_smooth_n_pts, outlier_threshold = config.outlier_thresh)
            bt_centers = list(accumulate(bt_centers, lambda x, y: y or x))
            bt_centers = [c if c is not None else (frame_shape[1] // 2, frame_shape[0] // 2)
                          for c in bt_centers]
                    
            subset_cropped_frames = get_cropped_frames(frames, bt_centers, config)
            save_centers += centers
            for save_frame in subset_cropped_frames:
                vid_writer.write(save_frame)
#             cropped_frames += subset_cropped_frames

            break
                
    cap.release()
    vid_writer.release()

def main(exp, config, resized_path : str):
    exp.test_conf = config.conf
    exp.nmsthre = config.nms
    
    model = exp.get_model()
    logger.info("Model Summary: {}".format(get_model_info(model, exp.test_size)))
    if config.device == "gpu":
        model.cuda().half()
    model.eval()

    ckpt_file = config.yolo_model_ckpt
    logger.info("loading checkpoint")
    ckpt = torch.load(ckpt_file, map_location="cpu")
    # load the model state dict
    model.load_state_dict(ckpt["model"])
    logger.info("loaded checkpoint done.")

    predictor = Predictor(
        model, exp, ("ball",), None, None,
        config.device
    )
    get_ball(predictor, config, resized_path)

In [5]:
class Ball_Track_Config:
    def __init__(self):
        self.video_path = './9f4df856_0.mp4'
        self.yolo_model_ckpt = './models/ball_tracking_model.pth'
        self.exp_file = './models/yolox_exp.py'
        self.nms = 0.5
        self.conf = 0.6
        self.device = "cpu"
        
        # ball tracking
        self.outlier_thresh = 20
        self.bt_smooth_n_pts = 2
        self.bt_batch_size = 64

        #cropping
        self.crop_frame_size = [256, 256]

['./data/clips/538438_0.mp4',
 './data/clips/a9f16c_6.mp4',
 './data/clips/c01561_4.mp4',
 './data/clips/121364_9.mp4']

In [25]:

!mkdir tracking_outputs
config = Ball_Track_Config()
for vid in glob.glob('./data/clips/*'):
    # if ('019d5b34_0.mp4' in vid) | ('019d5b34_1.mp4' in vid):
    exp = get_exp(config.exp_file, None)
    main(exp, config, vid)
torch.cuda.empty_cache()

mkdir: tracking_outputs: File exists


2022-11-09 12:49:54.689 | INFO     | __main__:main:113 - loading checkpoint
2022-11-09 12:49:56.437 | INFO     | __main__:main:117 - loaded checkpoint done.


./data/clips/121364_7.mp4
Saving to ./tracking_outputs/121364_7.mp4


data/             models/           submit.ipynb
eval/             src/              tracking_outputs/


In [38]:
torch.cuda.empty_cache()

In [12]:
%cp -r /kaggle/input/eventpreds4/ /kaggle/working/eventpreds4
!cp /kaggle/working/eventpreds4/pkgs/iopath-0.1.10.whl /kaggle/working/eventpreds4/pkgs/iopath-0.1.10.tar.gz
!cp /kaggle/working/eventpreds4/pkgs/fvcore-0.1.5.post20220512.whl /kaggle/working/eventpreds4/pkgs/fvcore-0.1.5.post20220512.tar.gz
!pip install /kaggle/working/eventpreds4/pkgs/iopath-0.1.10.tar.gz
!pip install /kaggle/working/eventpreds4/pkgs/av-9.2.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/working/eventpreds4/pkgs/fvcore-0.1.5.post20220512.tar.gz

Processing /kaggle/working/eventpreds4/pkgs/iopath-0.1.10.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31549 sha256=49090219e8ffd484fcc551bedeba428b38b7bbbc453ca9c499da36cdaa19aca9
  Stored in directory: /root/.cache/pip/wheels/21/ee/5b/b04f1861559da764b71860f05892744fba9f3684f8e4261d70
Successfully built iopath
Processing /kaggle/working/eventpreds4/pkgs/av-9.2.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/working/eventpreds4/pkgs/fvcore-0.1.5.post20220512.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.5.post20220512-py3-none-any.whl size=61288 sha256=0a5708571ef5ae6caf22742eec76ac2648c9930ce8f7ccfe265a11e1e911bd00
  Stored in directory: /root/.cache/pip/wheels/91/c8/85/7743398b7036cd1f332790e0de734227ac0e18ba0f297d743a
Successfully built fvcore


In [13]:
import torch
from tqdm import tqdm
from importlib import import_module
import os
import shutil
from glob import glob
import pytorchvideo
from pytorchvideo.data.ava import AvaLabeledVideoFramePaths
import torch.nn as nn

from src.SlowFast.data_loader.data_generator import DataLoader
from src.SlowFast.utils.training_utils import ModelCheckpoint
from src.SlowFast.utils.data_utils import *
import pickle as pkl

In [23]:
import torch

class SlowFast_Config:

    def __init__(self):
        self.resume_training = False

        self.loss_fn = 'CrossEntropyLoss'
        self.metric = 'AccMetric'
        self.architecture = 'slowfast'

        self.num_epochs = 200
        self.epoch_steps = 1600
        self.batch_size = 16
        self.grad_accum = 1
        self.learning_rate = 0.001
        self.window_len = 100  # number of frames in each window sent to model
        self.prediction_len = 24  # number of frames for which the model will predict events
        self.label_ratios = [0.28, 0.55, 0.12, 0.05]
        self.tolerances = [3, 5, 3, 0]
        self.img_size = 128
        self.size_fact = 256 // self.img_size
        self.pred_jump = 24

        self.data_mean = [0.45, 0.45, 0.45]
        self.data_std = [0.225, 0.225, 0.225]
        self.slow_fast_alpha = 4

        # augmentation
        self.aug = True
        self.aug_hflip_p = 0.5
        self.aug_scale = [0.7, 0.9]
        self.aug_scale_p = 0.2

        self.vid_paths = '/kaggle/working/tracking_outputs/'
        self.label_csv = 'train.csv'

        self.train_vids = ['3c993bd2_0', '3c993bd2_1', '1606b0e6_0', '1606b0e6_1',
                           'cfbe2e94_0', 'cfbe2e94_1', '35bd9041_0', '35bd9041_1', '4ffd5986_0']
        self.val_vids = ['407c5a9e_1', 'ecf251d4_0', '9a97dae4_1']
        self.labels = ['challenge', 'play', 'throwin']
        self.label_dict = {v: i for i, v in enumerate(self.labels)}


        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        
class SubmissionGenerator:
    def __init__(self, config, video_paths):
        self.config = config
        self.terminal_width = shutil.get_terminal_size((80, 20)).columns

        # Model
        print(f' Model: {self.config.architecture} '.center(self.terminal_width, '*'), end='\n\n')
        model_type = import_module('models.' + self.config.architecture)
        create_model = getattr(model_type, 'create_model')
        self.model = create_model(self.config)

        self.model = create_model(self.config)
        # print(self.model)
        self.config.fold = None
        model_checkpoint = ModelCheckpoint(config=self.config, weight_dir='../')
        self.model, _, _, _ = model_checkpoint.load(self.model, load_best=True)

        print(f' Loading data '.center(self.terminal_width, '*'))

        self.video_paths = video_paths

    def _generate_submission_file_model(self, model, outfile_prefix=''):
        print(f' Predict '.center(self.terminal_width, '*'))
        model.eval()
        start_int = (self.config.window_len - self.config.prediction_len) // 2

        vid_preds = {}
        for vid in self.video_paths:
            print(vid)

            data_loader = DataLoader(self.config)
            test_loader = data_loader.create_test_loader(vid)

            pred_range = test_loader.dataset.pred_range
            preds = np.zeros((int(test_loader.dataset.duration * 25), 4))
            sta = (self.config.window_len - self.config.prediction_len) // 2
            norm_factor = self.config.prediction_len // self.config.pred_jump

            for i, [x, idx] in enumerate(tqdm(test_loader)):
                x = [inp.to(self.config.device) for inp in x]
                idx = idx.data.numpy()

                with torch.autocast('cuda'):
                    pred = nn.functional.softmax(model(x), 1).data.cpu().numpy()
                    pred = pred[:, :, start_int:start_int + self.config.prediction_len]

                for b_pred_idx, b_pred in zip(idx, pred):
                    prev = preds[sta:sta+b_pred.shape[1]]
                    preds[sta:sta+b_pred.shape[1]] = prev + (b_pred.T[:prev.shape[0]] / norm_factor)
                    sta += self.config.pred_jump
        
            vid_preds[vid[vid.rfind('/') + 1:-4]] = preds
            
        return vid_preds

        # predictions = np.concatenate(preds, 0).transpose(2, 0, 1).reshape(-1, 28)
        # sample_submission = pd.read_csv('../data/sample_submission_uncertainty.csv')
        #
        # # Merge with sample_submission by using series ids
        # pred_ids = np.concatenate([[series_id[series_id.find('_') + 1:] + '_' + str(q).ljust(5, '0')
        #                             for series_id in self.agg_ids] for q in self.quantiles])
        # predictions_df = pd.DataFrame(np.hstack([pred_ids.reshape(-1, 1), predictions]),
        #                               columns=['id'] + [f'F{i + 1}' for i in range(28)])
        # sample_submission = sample_submission[['id']].merge(predictions_df, how='left',
        #                                                     left_on=sample_submission.id.str[:-11], right_on='id')
        # sample_submission['id'] = sample_submission['id_x']
        # del sample_submission['id_x'], sample_submission['id_y']
        #
        # # Export
        # sample_submission.to_csv(f'{self.sub_dir}/{outfile_prefix}submission.csv.gz', compression='gzip', index=False,
        #                          float_format='%.3g')

    def generate_submission_file(self):
        return self._generate_submission_file_model(self.model)
    
def find_labels(preds, vid):
    last_lab = None
    same_preds_probs = []
    same_preds_start_idx = 0
    vid_labs = []
    for idx, pr in enumerate(preds[38:-40].argmax(1)): 
        if last_lab is None:
            if pr != 3:
                last_lab = pr
                same_preds_probs.append(preds[idx + 38, pr])
                same_preds_start_idx = idx + 38
        else:
            if (pr == last_lab):
                same_preds_probs.append(preds[idx + 38, pr])
            else:
                if len(same_preds_probs) > 1:
                    pred_pos = np.average(np.arange(len(same_preds_probs)), weights=same_preds_probs)
                    # prob = preds[int(np.floor(pred_pos)) + same_preds_start_idx, last_lab]
                    prob = max(same_preds_probs)
                    pred_pos = (same_preds_start_idx + pred_pos) * 0.04
                    vid_labs.append([vid, pred_pos, config.labels[last_lab], prob])
                    same_preds_probs = []
                    same_preds_start_idx = 0

                    if pr != 3:
                        last_lab = pr
                        same_preds_probs.append(preds[idx + 38, pr])
                        same_preds_start_idx = idx + 38
                    else:
                        last_lab = None
                else:
                    if pr != 3:
                        last_lab = pr
                        same_preds_probs.append(preds[idx + 38, pr])
                        same_preds_start_idx = idx + 38
                    else:
                        last_lab = None
                    
    return vid_labs

In [16]:
config = SlowFast_Config()
import glob
vid_paths = glob.glob('./tracking_outputs/*')
predictor = SubmissionGenerator(config, vid_paths)
vid_preds = predictor.generate_submission_file()

******************************* Model: slowfast ********************************

********************************* Loading data *********************************
*********************************** Predict ************************************
/kaggle/working/ball-track/tracking_outputs/019d5b34_1.mp4


100%|██████████| 2/2 [00:05<00:00,  2.94s/it]


/kaggle/working/ball-track/tracking_outputs/019d5b34_0.mp4


100%|██████████| 2/2 [00:05<00:00,  2.72s/it]


In [24]:
sub_data = []
for vid, preds in vid_preds.items():
    vid_labs = find_labels(preds, vid)
    sub_data += vid_labs

In [27]:
sub_data

[['019d5b34_1', 5.351899254562475, 'play', 0.9138230681419373],
 ['019d5b34_1', 8.493307499301903, 'play', 0.9740211367607117],
 ['019d5b34_1', 10.279594628383885, 'play', 0.524573028087616],
 ['019d5b34_1', 13.717983955411503, 'play', 0.8791025876998901],
 ['019d5b34_1', 17.8988588060504, 'play', 0.4833300709724426],
 ['019d5b34_1', 19.122520057823102, 'play', 0.6615259647369385],
 ['019d5b34_1', 23.800946161953252, 'play', 0.9545438885688782],
 ['019d5b34_1', 24.919404908167778, 'play', 0.8766966462135315],
 ['019d5b34_1', 25.595996098606143, 'play', 0.5223039388656616],
 ['019d5b34_1', 26.5182817485419, 'play', 0.5261824727058411],
 ['019d5b34_0', 2.9386480435913938, 'play', 0.7140445113182068],
 ['019d5b34_0', 16.861001777835405, 'play', 0.5850505232810974],
 ['019d5b34_0', 19.10974876262489, 'play', 0.8750974535942078],
 ['019d5b34_0', 22.622186513770803, 'play', 0.8794339299201965],
 ['019d5b34_0', 25.44351352180087, 'play', 0.890841007232666],
 ['019d5b34_0', 27.620485138862698,

In [ ]:
import pandas as pd
sub_df = pd.DataFrame(sub_data, columns=['video_id', 'time', 'event', 'score'])
# Export
sub_df.to_csv("/kaggle/working/submission.csv", index=False)